In [3]:
import os
import itertools
import numpy as np
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output
import nglview as nv

# ASE
from ase.io import read, write
from ase.build import stack,cut

# Matlantis / PFP
from matlantis_features.atoms import MatlantisAtoms
from matlantis_features.features.common.opt import FireLBFGSASEOptFeature
from matlantis_features.utils.calculators import pfp_estimator_fn
from pfp_api_client.pfp.estimator import EstimatorCalcMode

class TrajectoryCleanup:
    """
    .trajファイルを分析し、.xyzに変換してクリーンアップするクラス。
    """
    def __init__(self, directory):
        self.directory = Path(directory)
        print("--- STEP 1: Trajectoryファイルの分析と整理を開始 ---")
        if not self.directory.is_dir():
            print(f"🛑 エラー: ディレクトリ '{self.directory}' が見つかりません。")
            self.files_to_process = []
        else:
            self.files_to_process = list(self.directory.glob("*.traj"))
            print(f"🔍 '{self.directory}' から {len(self.files_to_process)} 個の .traj ファイルを発見。")
    def process_file(self, traj_path):
        """
        1つの.trajファイルを処理する。
        [修正] この関数全体をtry...exceptで囲み、エラー耐性を向上。
        """
        try:
            # ファイルサイズを確認
            file_size = traj_path.stat().st_size
            if file_size == 0:
                print(f"  -> 🗑️  0KBファイルを削除します: {traj_path.name}")
                os.remove(traj_path)
                return

            # 正常なファイルは変換
            base_name = traj_path.stem
            initial_xyz_path = self.directory / f"{base_name}_initial.xyz"
            final_xyz_path = self.directory / f"{base_name}_final.xyz"
            
            print(f"  -> 🔄 変換中: {traj_path.name}")
            
            # Trajectoryから初期構造と最終構造を読み込み
            atoms_list = read(traj_path, index=":")
            if not atoms_list:
                print(f"  -> ❌ 読み込みエラー（空のTrajectory）、ファイルを削除します: {traj_path.name}")
                os.remove(traj_path)
                return

            # .xyzファイルとして保存
            write(str(initial_xyz_path), atoms_list[0])
            write(str(final_xyz_path), atoms_list[-1])
            
            # 元の.trajファイルを削除
            os.remove(traj_path)
            print(f"  -> ✔️ 変換成功、元のファイルを削除しました: {traj_path.name}")

        except Exception as e:
            # 予期せぬエラーが発生した場合、ログを出力して処理を続行
            print(f"  -> ❌ 重大なエラーが発生したため、このファイルはスキップします ({traj_path.name}): {e}")

    def run(self):
        """全ての対象ファイルに対して処理を実行する。"""
        if not self.files_to_process:
            print(" -> 処理対象の.trajファイルがないため、スキップします。")
            return
        for traj_file in self.files_to_process:
            self.process_file(traj_file)
        print("\n✨ STEP 1 完了。")


# --- メイン実行部 ---
if __name__ == "__main__":
    # ===== 設定項目 =====
    # 1. Trajectoryファイルが保存されているディレクトリ
    OPTIMIZATION_OUTPUT_DIR = "/home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces"
    
    # 2. 最適化した界面の保存先
    INTERFACES_DIR = "/home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces"
    # ====================

    clear_output(wait=True)
    
    # STEP 1: メモリ管理 (traj -> xyz変換 & 削除)
    cleanup = TrajectoryCleanup(OPTIMIZATION_OUTPUT_DIR)
    cleanup.run()
    

--- STEP 1: Trajectoryファイルの分析と整理を開始 ---
🔍 '/home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces' から 343 個の .traj ファイルを発見。
  -> 🔄 変換中: Interface_AlF3_on_NMC111_CotoNi_50_100_(1, 0, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_CotoNi_50_100_(1, 0, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_CotoNi_80_001_(0, 0, 1)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_CotoNi_80_001_(0, 0, 1)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_on_NMC111_CotoNi_50_012_(0, 1, 2)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_pristine_001_(0, 0, 1)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_pristine_001_(0, 0, 1)_trimmed.traj
  -> 🔄 変換中: Interface_Al_100_(1, 0, 0)_on_NMC111_CotoNi_50_110_(1, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_100_(1, 0, 0)_on_NMC111_CotoNi_50_110_(1, 1, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al_111_(1, 1, 1)_on_NMC111_sel